In [1]:
import pandas as pd

In [2]:
statements = pd.read_csv('../data/processed_data/재무지표.csv')
listed = pd.read_csv('../data/processed_data/개별재무지표.csv')

---
### 전체에 대해서 값이 없는 행(기업) 탐색

In [3]:
# 모든 컬럼에서 값이 없는(결측치) 행을 찾기
empty_rows = statements[statements.iloc[:, 9:-1].isnull().all(axis=1)]

print(len(empty_rows.drop_duplicates('회사명')['회사명']))
print('------------------------------------------')
print(empty_rows.drop_duplicates('회사명')['회사명'])

35
------------------------------------------
7          (주)대호에이엘
12       (주)신화다이나믹스
74          백광산업(주)
78      삼성바이오로직스(주)
95        코스모신소재(주)
98         코스모화학(주)
105       한국전력기술(주)
114      한솔아트원제지(주)
131           (주)광무
144          (주)뉴보텍
156        (주)디지털대성
158          (주)메지온
165           (주)삼일
167           (주)상보
173         (주)서울제약
195          (주)시큐브
211           (주)씨젠
227        (주)에스에스알
246         (주)와이어블
274      (주)제이테크놀로지
286     (주)케이바이오컴퍼니
291    (주)케이에스인더스트리
296     (주)케이지모빌리언스
298      (주)케이지이니시스
306        (주)타이거일렉
309       (주)티에스넥스젠
328        (주)한진피앤씨
334         경남제약(주)
339       대한그린파워(주)
341          더라미(주)
347         디에스티(주)
350        삼영이엔씨(주)
356        씨앤티85(주)
369       우양에이치씨(주)
380       한솔아이원스(주)
Name: 회사명, dtype: object


In [4]:
print(empty_rows.shape)
print(empty_rows.drop_duplicates('회사명').shape)

(89, 194)
(35, 194)


In [5]:
# empty_rows.to_csv('재무지표null.csv',index=False, encoding='UTF-8-sig')

In [6]:
# 결측치 기업들 feature 컬럼 제외
df = empty_rows.iloc[:, :9]
df['label'] = empty_rows['label']

df.reset_index(drop=True, inplace=True)
df.head()

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,label
0,(주)대호에이엘,69460,2014,1,24,32402,1차 금속 제조업,2002-11-11,NaN,1
1,(주)신화다이나믹스,1770,2012,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
2,(주)신화다이나믹스,1770,2013,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
3,(주)신화다이나믹스,1770,2014,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1
4,(주)신화다이나믹스,1770,2015,1,24,32401,1차 금속 제조업,1988-11-23,NaN,1


### 결측치 기업 개별재무비율로 대체

In [7]:
import numpy as np

In [8]:
statements
listed

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,현금및현금성자산(*)(IFRS)(천원),...,외환차손(IFRS)(천원).1,외화환산손실(IFRS)(천원).1,지분법적용투자주식손상차손(IFRS)(천원),기부금(IFRS)(천원),투자자산처분손실(IFRS)(천원),보상비(IFRS)(천원),법인세추납액(IFRS)(천원),전기오류수정손실(IFRS)(천원),법인세비용(IFRS)(천원),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,5547269.0,...,2136822.0,2881977.0,0.0,0.0,0.0,0.0,0.0,0.0,1032733.0,1
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,7869589.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9539579.0,1
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,1514457.0,...,2365382.0,2147683.0,0.0,0.0,0.0,0.0,0.0,0.0,-4431729.0,1
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,323571800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56151157.0,0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,324046180.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-127568854.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9892897.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553201.0,1
385,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,5510953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1544374.0,1
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,12307845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2093131.0,1
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9272814.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2536049.0,1


In [9]:
listed = listed.rename(columns={'공구,기구,비품(IFRS)(천원)': '공구', '조리법, 공식, 모형, 설계 및 시제품(IFRS)(천원)': '조리법', '유형,리스자산처분이익(IFRS)(천원)': '유형'})

In [10]:
def clean_column_name(col_name):
    col_name = col_name.replace('(*)', '').replace('(원)', '').replace('(백만원)', '')
    col_name = col_name.replace('[제조]', '')
    col_name = col_name.replace('(천원)', '')
    col_name = col_name.replace('(IFRS연결)', '(IFRS)')
    # col_name = col_name.replace('.1', '')

    return col_name.strip()  


listed.columns = [clean_column_name(col) for col in listed.columns]
statements.columns = [clean_column_name(col) for col in statements.columns]

In [12]:
statements['공구']

0       9311662.0
1             0.0
2       9465745.0
3      80829000.0
4      80245000.0
          ...    
384           NaN
385           NaN
386    13004596.0
387    13598336.0
388    13859285.0
Name: 공구, Length: 389, dtype: float64

In [13]:
listed['공구']

0       5699084.0
1             0.0
2       5929846.0
3      51844000.0
4      49423000.0
          ...    
384    11539743.0
385    13424068.0
386    13004596.0
387    13598336.0
388    13859285.0
Name: 공구, Length: 389, dtype: float64

In [14]:
statements.replace(0, np.nan, inplace=True)

In [15]:
statements.set_index(['회사명', '거래소코드', '회계년도'], inplace=True)
listed.set_index(['회사명', '거래소코드', '회계년도'], inplace=True)

In [16]:
df1, df2 = statements.align(listed, join='left')

In [17]:
result = df1.fillna(df2)

In [18]:
result = result.reset_index()

In [19]:
result

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,현금및현금성자산(IFRS),...,외환차손(IFRS).1,외화환산손실(IFRS).1,지분법적용투자주식손상차손(IFRS),기부금(IFRS),투자자산처분손실(IFRS),보상비(IFRS),법인세추납액(IFRS),전기오류수정손실(IFRS),법인세비용(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,10376890.0,...,2136822.0,2881977.0,0.0,0.0,0.0,0.0,0.0,0.0,184992.0,1.0
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,15192647.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7248212.0,1.0
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,6564156.0,...,2365382.0,2147683.0,0.0,0.0,0.0,0.0,0.0,0.0,-696299.0,1.0
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,388440274.0,...,38000000.0,23052000.0,0.0,0.0,0.0,0.0,0.0,0.0,54918791.0,0.0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,409596263.0,...,34514000.0,23702000.0,0.0,0.0,0.0,0.0,0.0,0.0,-121688653.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9892897.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553201.0,1.0
385,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,5510953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1544374.0,1.0
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,12310161.0,...,821838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2093131.0,1.0
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9274990.0,...,799836.0,2359896.0,0.0,0.0,0.0,0.0,0.0,0.0,2536049.0,1.0


In [20]:
result.columns[result.isnull().sum() > 0]

Index(['상장폐지일', '상각후원가측정 유가증권(IFRS)', '임대(주택)자산(IFRS)', '임차개량자산(IFRS)',
       '배출권자산(IFRS)', '당기법인세부채(미지급법인세)(IFRS)', '이자수익(IFRS)', '배당금수익(IFRS)',
       '사채상환이익(IFRS)', '이자비용(IFRS)', '기타대손상각비(IFRS)'],
      dtype='object')

In [21]:
listed = listed.reset_index()
listed.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '현금및현금성자산(IFRS)',
       ...
       '외환차손(IFRS).1', '외화환산손실(IFRS).1', '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)',
       '투자자산처분손실(IFRS)', '보상비(IFRS)', '법인세추납액(IFRS)', '전기오류수정손실(IFRS)',
       '법인세비용(IFRS)', 'label'],
      dtype='object', length=194)

In [22]:
statements = statements.reset_index()
statements.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '현금및현금성자산(IFRS)',
       ...
       '외환차손(IFRS).1', '외화환산손실(IFRS).1', '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)',
       '투자자산처분손실(IFRS)', '보상비(IFRS)', '법인세추납액(IFRS)', '전기오류수정손실(IFRS)',
       '법인세비용(IFRS)', 'label'],
      dtype='object', length=194)

In [23]:
cols_to_front = ['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일']
cols_to_end = ['label']
new_columns_order = cols_to_front + [col for col in statements.columns if col not in (cols_to_front + cols_to_end)] + cols_to_end
result = result[new_columns_order]

In [24]:
result

,회사명,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,산업명,상장일,상장폐지일,현금및현금성자산(IFRS),...,외환차손(IFRS).1,외화환산손실(IFRS).1,지분법적용투자주식손상차손(IFRS),기부금(IFRS),투자자산처분손실(IFRS),보상비(IFRS),법인세추납액(IFRS),전기오류수정손실(IFRS),법인세비용(IFRS),label
0,(주)다이나믹디자인,145210,2015,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,10376890.0,...,2136822.0,2881977.0,0.0,0.0,0.0,0.0,0.0,0.0,184992.0,1.0
1,(주)다이나믹디자인,145210,2016,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,15192647.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7248212.0,1.0
2,(주)다이나믹디자인,145210,2017,1,29,32902,기타 기계 및 장비 제조업,2015-03-19,NaN,6564156.0,...,2365382.0,2147683.0,0.0,0.0,0.0,0.0,0.0,0.0,-696299.0,1.0
3,(주)대우건설,47040,2012,1,41,64101,종합 건설업,2001-03-23,NaN,388440274.0,...,38000000.0,23052000.0,0.0,0.0,0.0,0.0,0.0,0.0,54918791.0,0.0
4,(주)대우건설,47040,2013,1,41,64101,종합 건설업,2001-03-23,NaN,409596263.0,...,34514000.0,23702000.0,0.0,0.0,0.0,0.0,0.0,0.0,-121688653.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,한솔아이원스(주),114810,2017,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9892897.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,553201.0,1.0
385,한솔아이원스(주),114810,2018,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,5510953.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1544374.0,1.0
386,한솔아이원스(주),114810,2019,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,12310161.0,...,821838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2093131.0,1.0
387,한솔아이원스(주),114810,2020,5,29,32902,기타 기계 및 장비 제조업,2013-02-07,NaN,9274990.0,...,799836.0,2359896.0,0.0,0.0,0.0,0.0,0.0,0.0,2536049.0,1.0


In [26]:
# result.to_csv('재무지표_개별로대체_1002.csv',index=False,encoding='UTF-8-sig')

전체가 na인 컬럼들 제거

In [30]:
# result_cleaned = result.dropna(axis=1, how='all')
# result_cleaned

In [27]:
cols_to_drop = result.columns[result.isna().all()]
result_cleaned = result.drop(columns=cols_to_drop)

In [28]:
cols_to_drop

Index([], dtype='object')

채워줬음에도 전체가 0인 컬럼들 제거하기

In [29]:
cols_to_drop2 = result.columns[(result==0).all()]
df_cleaned = result.drop(columns=cols_to_drop2)

In [30]:
df = pd.DataFrame(cols_to_drop2)

In [31]:
# df.to_csv("0값제거컬럼.csv")

In [32]:
cols_to_drop2

Index(['리스개량자산(IFRS)', '유형탐사평가자산(IFRS)', '임대주택용지(IFRS)', '미완성임대주택(IFRS)',
       '임대주택토지(IFRS)', '완성임대주택(IFRS)', '임대주택(IFRS)', '기타임대주택자산(IFRS)',
       '제호와 출판표제(IFRS)', '조리법', '*특수관계자장기매출채권(IFRS)', '장기분양미수금(IFRS)',
       '*특수관계자 장기미수금(IFRS)', '장기임대미수금(IFRS)', '장기임대주택채권(IFRS)',
       '(추정공사손실)(IFRS)', '배당건설이자(IFRS)', '파생상품평가이익(IFRS)', '파생상품평가손실(IFRS)',
       '퇴직급여적립금(IFRS)', '(대체)(IFRS)', '미분양주택관리비(IFRS)', '투자자산처분이익(IFRS)', '유형',
       '무형자산처분이익(IFRS)', '법인세환급액(IFRS)', '재고자산감모손실(IFRS)',
       '지분법적용투자주식손상차손(IFRS)', '기부금(IFRS)', '투자자산처분손실(IFRS)', '법인세추납액(IFRS)',
       '전기오류수정손실(IFRS)'],
      dtype='object')

In [33]:
# df_cleaned.to_csv('../data/processed_data/재무지표_열제거_1002.csv', index=False, encoding='UTF-8-sig')

행이 모두 nan 값이면 지우는 코드

In [34]:

# 9번째 컬럼부터 마지막 전까지 값이 모두 NaN인 행 찾기
rows_to_remove = df_cleaned[df_cleaned.iloc[:, 9:-1].isna().all(axis=1)]

# 삭제된 행의 회사명 저장
removed_company_names = rows_to_remove.iloc[:, 0]  # 0번째 컬럼 (회사명)

# 새로운 데이터프레임에서 지운 행 제거
rows_to_keep = df_cleaned[~df_cleaned.iloc[:, 9:-1].isna().all(axis=1)]

# 결과 출력
print("삭제된 행의 회사명:")
print(removed_company_names.tolist())
print("\n새로운 데이터프레임:")
print(rows_to_keep)


삭제된 행의 회사명:
[]

새로운 데이터프레임:
            회사명   거래소코드  회계년도  소속코드  통계청 한국표준산업분류 코드 10차(대분류)   산업코드  \
0    (주)다이나믹디자인  145210  2015     1                        29  32902   
1    (주)다이나믹디자인  145210  2016     1                        29  32902   
2    (주)다이나믹디자인  145210  2017     1                        29  32902   
3       (주)대우건설   47040  2012     1                        41  64101   
4       (주)대우건설   47040  2013     1                        41  64101   
..          ...     ...   ...   ...                       ...    ...   
384   한솔아이원스(주)  114810  2017     5                        29  32902   
385   한솔아이원스(주)  114810  2018     5                        29  32902   
386   한솔아이원스(주)  114810  2019     5                        29  32902   
387   한솔아이원스(주)  114810  2020     5                        29  32902   
388   한솔아이원스(주)  114810  2021     5                        29  32902   

                   산업명         상장일 상장폐지일  현금및현금성자산(IFRS)  ...  전기오류수정이익(IFRS)  \
0      기타 기계 및 장비 제조업   20

In [35]:
# 9번째 컬럼부터 마지막 전까지 값이 모두 0인 행 찾기
rows_to_remove = df_cleaned[(df_cleaned.iloc[:, 9:-1] == 0).all(axis=1)]

# 삭제된 행의 회사명 저장
removed_company_names = rows_to_remove.iloc[:, 0]  # 0번째 컬럼 (회사명)

# 새로운 데이터프레임에서 지운 행 제거
rows_to_keep = df_cleaned[~(df_cleaned.iloc[:, 9:-1] == 0).all(axis=1)]

# 결과 출력
print("삭제된 행의 회사명:")
print(removed_company_names.tolist())
print("\n새로운 데이터프레임:")
print(rows_to_keep)

삭제된 행의 회사명:
[]

새로운 데이터프레임:
            회사명   거래소코드  회계년도  소속코드  통계청 한국표준산업분류 코드 10차(대분류)   산업코드  \
0    (주)다이나믹디자인  145210  2015     1                        29  32902   
1    (주)다이나믹디자인  145210  2016     1                        29  32902   
2    (주)다이나믹디자인  145210  2017     1                        29  32902   
3       (주)대우건설   47040  2012     1                        41  64101   
4       (주)대우건설   47040  2013     1                        41  64101   
..          ...     ...   ...   ...                       ...    ...   
384   한솔아이원스(주)  114810  2017     5                        29  32902   
385   한솔아이원스(주)  114810  2018     5                        29  32902   
386   한솔아이원스(주)  114810  2019     5                        29  32902   
387   한솔아이원스(주)  114810  2020     5                        29  32902   
388   한솔아이원스(주)  114810  2021     5                        29  32902   

                   산업명         상장일 상장폐지일  현금및현금성자산(IFRS)  ...  전기오류수정이익(IFRS)  \
0      기타 기계 및 장비 제조업   20

In [36]:
df_cleaned.describe()

,거래소코드,회계년도,소속코드,통계청 한국표준산업분류 코드 10차(대분류),산업코드,현금및현금성자산(IFRS),유동금융자산(IFRS),매출채권 및 기타유동채권(IFRS),이연법인세자산(IFRS),상각후원가측정 유가증권(IFRS),...,전기오류수정이익(IFRS),채무면제이익(IFRS),보험차익(IFRS),이자비용(IFRS),기타대손상각비(IFRS),외환차손(IFRS).1,외화환산손실(IFRS).1,보상비(IFRS),법인세비용(IFRS),label
count,389.000000,389.000000,389.000000,389.000000,389.000000,3.890000e+02,3.890000e+02,3.890000e+02,3.890000e+02,1.360000e+02,...,389.000000,3.890000e+02,389.000000,3.570000e+02,3.880000e+02,3.890000e+02,3.890000e+02,389.000000,3.890000e+02,389.000000
mean,61060.848329,2015.241645,3.619537,36.161954,56498.365039,8.232927e+07,5.939977e+07,2.863112e+08,2.124664e+07,1.782881e+05,...,1451.727506,4.142142e+04,3169.185090,5.899429e+03,4.650685e+06,8.449598e+06,4.979738e+06,9301.506427,8.820598e+06,0.760925
std,66242.963571,2.113582,1.856787,17.630134,39051.146040,2.911515e+08,2.210259e+08,1.025248e+09,9.770319e+07,1.760871e+06,...,23047.758864,4.042075e+05,38110.962978,7.687665e+04,2.706963e+07,4.850450e+07,2.479776e+07,65152.672558,7.493840e+07,0.427068
min,720.000000,2012.000000,1.000000,10.000000,31201.000000,8.790000e+02,0.000000e+00,2.830360e+05,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,-7.367819e+06,0.000000e+00,0.000000e+00,0.000000,-9.506336e+08,0.000000
25%,27580.000000,2014.000000,1.000000,24.000000,32501.000000,2.503144e+06,1.051000e+06,9.056679e+06,0.000000e+00,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.697000e+03,1.000000e+00,0.000000,0.000000e+00,1.000000
50%,47040.000000,2015.000000,5.000000,29.000000,32902.000000,8.460020e+06,4.120500e+06,2.147539e+07,4.528100e+05,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,2.435350e+05,7.175700e+04,0.000000,3.019280e+05,1.000000
75%,69460.000000,2017.000000,5.000000,46.000000,74605.000000,3.063622e+07,1.714959e+07,6.569940e+07,4.709323e+06,0.000000e+00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,7.003280e+05,1.424510e+06,5.860770e+05,0.000000,3.403395e+06,1.000000
max,900100.000000,2021.000000,6.000000,91.000000,189102.000000,2.542381e+09,2.099634e+09,7.980449e+09,1.335461e+09,2.029100e+07,...,449505.000000,6.421716e+06,575056.000000,1.070870e+06,4.194660e+08,7.328480e+08,3.391720e+08,745141.000000,4.191244e+08,1.000000


## 컬럼별 0인것들의 개수

In [37]:
pd.set_option('display.max_columns', None)

In [38]:
pd.set_option('display.max_rows', None)

In [39]:
len(df_cleaned.columns) -9

153

In [40]:
na_count = (df_cleaned.isna()).sum()
na_count

회사명                               0
거래소코드                             0
회계년도                              0
소속코드                              0
통계청 한국표준산업분류 코드 10차(대분류)          0
산업코드                              0
산업명                               0
상장일                               0
상장폐지일                           331
현금및현금성자산(IFRS)                    0
유동금융자산(IFRS)                      0
매출채권 및 기타유동채권(IFRS)               0
이연법인세자산(IFRS)                     0
상각후원가측정 유가증권(IFRS)              253
매출채권(IFRS)                        0
미수금(IFRS)                         0
단기대여금(IFRS)                       0
선급금(IFRS)                         0
기타유동자산(IFRS)                      0
상품(IFRS)                          0
제품(IFRS)                          0
반제품(IFRS)                         0
재공품(IFRS)                         0
원재료(부재료)(IFRS)                    0
저장품(소모품)(IFRS)                    0
미착품(IFRS)                         0
완성건물(IFRS)                        0
용지(IFRS)                    

In [41]:
na_count[na_count > 0]

상장폐지일                    331
상각후원가측정 유가증권(IFRS)       253
임대(주택)자산(IFRS)            33
임차개량자산(IFRS)              33
배출권자산(IFRS)              253
당기법인세부채(미지급법인세)(IFRS)    192
이자수익(IFRS)                11
배당금수익(IFRS)               19
사채상환이익(IFRS)              33
이자비용(IFRS)                32
기타대손상각비(IFRS)              1
dtype: int64

In [42]:
zero_count = (df_cleaned==0).sum()

In [43]:
print(zero_count)

회사명                               0
거래소코드                             0
회계년도                              0
소속코드                              0
통계청 한국표준산업분류 코드 10차(대분류)          0
산업코드                              0
산업명                               0
상장일                               0
상장폐지일                             0
현금및현금성자산(IFRS)                    0
유동금융자산(IFRS)                     17
매출채권 및 기타유동채권(IFRS)               0
이연법인세자산(IFRS)                   161
상각후원가측정 유가증권(IFRS)              132
매출채권(IFRS)                        4
미수금(IFRS)                         8
단기대여금(IFRS)                      91
선급금(IFRS)                         8
기타유동자산(IFRS)                      1
상품(IFRS)                        129
제품(IFRS)                        117
반제품(IFRS)                       329
재공품(IFRS)                       168
원재료(부재료)(IFRS)                   85
저장품(소모품)(IFRS)                  223
미착품(IFRS)                       222
완성건물(IFRS)                      370
용지(IFRS)                    

In [44]:
df_cleaned.columns

Index(['회사명', '거래소코드', '회계년도', '소속코드', '통계청 한국표준산업분류 코드 10차(대분류)', '산업코드',
       '산업명', '상장일', '상장폐지일', '현금및현금성자산(IFRS)',
       ...
       '전기오류수정이익(IFRS)', '채무면제이익(IFRS)', '보험차익(IFRS)', '이자비용(IFRS)',
       '기타대손상각비(IFRS)', '외환차손(IFRS).1', '외화환산손실(IFRS).1', '보상비(IFRS)',
       '법인세비용(IFRS)', 'label'],
      dtype='object', length=162)

In [45]:
filtered_zero_count = zero_count[zero_count < 10]
filtered_zero_count

회사명                         0
거래소코드                       0
회계년도                        0
소속코드                        0
통계청 한국표준산업분류 코드 10차(대분류)    0
산업코드                        0
산업명                         0
상장일                         0
상장폐지일                       0
현금및현금성자산(IFRS)              0
매출채권 및 기타유동채권(IFRS)         0
매출채권(IFRS)                  4
미수금(IFRS)                   8
선급금(IFRS)                   8
기타유동자산(IFRS)                1
미지급금(IFRS)                  7
당기법인세부채(미지급법인세)(IFRS)       0
보통주자본금(IFRS)                0
미처분이익잉여금(결손금)(IFRS)         0
매출액(수익)(IFRS)               0
종업원 급여비용(IFRS)              3
급여(IFRS)                    3
복리후생비(IFRS)                 9
감가상각비(IFRS)                 7
dtype: int64

In [46]:
# filtered_zero_count.to_csv("결측치 10개미만.csv")

In [47]:
len(filtered_zero_count)-9

15

In [48]:
filtered_zero_count2 = zero_count[zero_count < 50]
filtered_zero_count2

회사명                          0
거래소코드                        0
회계년도                         0
소속코드                         0
통계청 한국표준산업분류 코드 10차(대분류)     0
산업코드                         0
산업명                          0
상장일                          0
상장폐지일                        0
현금및현금성자산(IFRS)               0
유동금융자산(IFRS)                17
매출채권 및 기타유동채권(IFRS)          0
매출채권(IFRS)                   4
미수금(IFRS)                    8
선급금(IFRS)                    8
기타유동자산(IFRS)                 1
토지(IFRS)                    46
건물 및 부속설비(IFRS)             41
매입채무(IFRS)                  11
미지급금(IFRS)                   7
미지급비용(IFRS)                 12
선수금(IFRS)                   36
당기법인세부채(미지급법인세)(IFRS)        0
기타유동부채(IFRS)                19
보통주자본금(IFRS)                 0
주식발행초과금(IFRS)               20
미처분이익잉여금(결손금)(IFRS)          0
매출액(수익)(IFRS)                0
종업원 급여비용(IFRS)               3
급여(IFRS)                     3
퇴직급여(IFRS)                  12
복리후생비(IFRS)                  9
여비교통비(IF

In [49]:
len(filtered_zero_count2) -9

26

In [50]:
filtered_zero_count2 = zero_count[zero_count < 150]
filtered_zero_count2

회사명                           0
거래소코드                         0
회계년도                          0
소속코드                          0
통계청 한국표준산업분류 코드 10차(대분류)      0
산업코드                          0
산업명                           0
상장일                           0
상장폐지일                         0
현금및현금성자산(IFRS)                0
유동금융자산(IFRS)                 17
매출채권 및 기타유동채권(IFRS)           0
상각후원가측정 유가증권(IFRS)          132
매출채권(IFRS)                    4
미수금(IFRS)                     8
단기대여금(IFRS)                  91
선급금(IFRS)                     8
기타유동자산(IFRS)                  1
상품(IFRS)                    129
제품(IFRS)                    117
원재료(부재료)(IFRS)               85
토지(IFRS)                     46
건물 및 부속설비(IFRS)              41
기계장치(IFRS)                   60
차량운반구(IFRS)                  68
공구                           79
회원권(IFRS)                   149
배출권자산(IFRS)                 136
매입채무(IFRS)                   11
미지급금(IFRS)                    7
미지급비용(IFRS)                  12
선수금(IFRS

In [52]:
len(filtered_zero_count2)-10

53

In [53]:
# filtered_zero_count2.to_csv("결측치 150개미만.csv")

In [62]:
df3 = pd.read_csv("결측치 150개미만.csv", index_col=0)
df3.reset_index(False, inplace=True)

In [66]:
df3_col = list(df3['index'])
df3_col

['회사명',
 '거래소코드',
 '회계년도',
 '소속코드',
 '통계청 한국표준산업분류 코드 10차(대분류)',
 '산업코드',
 '산업명',
 '상장일',
 '상장폐지일',
 '현금및현금성자산(IFRS)',
 '유동금융자산(IFRS)',
 '매출채권 및 기타유동채권(IFRS)',
 '상각후원가측정 유가증권(IFRS)',
 '매출채권(IFRS)',
 '미수금(IFRS)',
 '단기대여금(IFRS)',
 '선급금(IFRS)',
 '기타유동자산(IFRS)',
 '상품(IFRS)',
 '제품(IFRS)',
 '원재료(부재료)(IFRS)',
 '토지(IFRS)',
 '건물 및 부속설비(IFRS)',
 '기계장치(IFRS)',
 '차량운반구(IFRS)',
 '공구',
 '회원권(IFRS)',
 '배출권자산(IFRS)',
 '매입채무(IFRS)',
 '미지급금(IFRS)',
 '미지급비용(IFRS)',
 '선수금(IFRS)',
 '당기법인세부채(미지급법인세)(IFRS)',
 '기타유동부채(IFRS)',
 '장기차입금(IFRS)',
 '보통주자본금(IFRS)',
 '주식발행초과금(IFRS)',
 '자기주식(IFRS)',
 '기타포괄손익누계액(IFRS)',
 '이익준비금(IFRS)',
 '미처분이익잉여금(결손금)(IFRS)',
 '매출액(수익)(IFRS)',
 '제품매출액(IFRS)',
 '기타수익(IFRS)',
 '종업원 급여비용(IFRS)',
 '급여(IFRS)',
 '퇴직급여(IFRS)',
 '복리후생비(IFRS)',
 '보험료(IFRS)',
 '여비교통비(IFRS)',
 '임차료(IFRS)',
 '접대비(IFRS)',
 '감가상각비(IFRS)',
 '광고선전비(IFRS)',
 '차량유지비(IFRS)',
 '연구개발비(IFRS)',
 '대손상각비(IFRS)',
 '기타(IFRS)',
 '이자수익(IFRS)',
 '외환차손(IFRS).1',
 '외화환산손실(IFRS).1',
 '법인세비용(IFRS)',
 'label']

In [69]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 5)

In [72]:
df_cleaned[df3_col].to_csv("../data/processed_data/재무지표_150개이상제거완.csv")